# Data Preprocessing cho Logistic Regression

Notebook này chứa các bước tiền xử lý dữ liệu để chuẩn bị cho mô hình Logistic Regression dự đoán Customer Churn.

#### Môi trường code

In [1]:
import sys
sys.executable

'/home/xv6/anaconda3/envs/min_ds-env/bin/python'

#### Import thư viện

In [2]:
# Import thư viện cần thiết
import numpy as np
import os
import sys

os.chdir("/home/xv6/Lab 2/Credit Card Customer Analysis")

project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.append(project_root)
    
import src.data_processing as dp

## 1. Load dữ liệu

In [3]:
# Load dữ liệu
data_path = "data/raw/BankChurners.csv"
data = dp.load_data(data_path)

# Preprocessing bước đầu: loại bỏ 2 cột cuối như đã phân tích
data_clean = dp.preprocess_data(data, exclude_last_n=2)

print(f"Số dòng: {data_clean.shape[0]}")
print(f"Số cột: {len(data_clean.dtype.names)}")

Số dòng: 10127
Số cột: 21


## 2. Tạo target variable cho Logistic Regression

In [4]:
# Sử dụng hàm tạo target binary từ module
target_binary = dp.create_binary_target(data_clean, 'Attrition_Flag', 'Attrited Customer')

print("Phân bố target:")
unique, counts = np.unique(target_binary, return_counts=True)
for val, count in zip(unique, counts):
    label = "Attrited Customer" if val == 1 else "Existing Customer"
    percentage = count / len(target_binary) * 100
    print(f"{label} (class {val}): {count:,} ({percentage:.1f}%)")

Phân bố target:
Existing Customer (class 0): 8,500 (83.9%)
Attrited Customer (class 1): 1,627 (16.1%)


## 3. Feature Selection và Engineering

In [5]:
# Feature Selection sử dụng các hàm có sẵn từ module
excluded_columns = ['CLIENTNUM', 'Attrition_Flag']  # ID và target

# Sử dụng hàm get_numeric_columns và get_categorical_columns từ module
numeric_features = dp.get_numeric_columns(data_clean, exclude_columns=excluded_columns)

categorical_features = dp.get_categorical_columns(data_clean)
categorical_features = [col for col in categorical_features if col not in excluded_columns]

# Tổng hợp feature columns
feature_columns = numeric_features + categorical_features

print("Các features sẽ được sử dụng:")
print("=" * 50)

print("NUMERIC FEATURES:")
for i, col in enumerate(numeric_features, 1):
    print(f"{i:2d}. {col}")

print(f"\nCATEGORICAL FEATURES:")
for i, col in enumerate(categorical_features, 1):
    print(f"{i:2d}. {col}")

print(f"\nTỔNG KẾT:")
print(f"- Tổng số features: {len(feature_columns)}")
print(f"- Numeric features: {len(numeric_features)}")
print(f"- Categorical features: {len(categorical_features)}")

Các features sẽ được sử dụng:
NUMERIC FEATURES:
 1. Customer_Age
 2. Dependent_count
 3. Months_on_book
 4. Total_Relationship_Count
 5. Months_Inactive_12_mon
 6. Contacts_Count_12_mon
 7. Credit_Limit
 8. Total_Revolving_Bal
 9. Avg_Open_To_Buy
10. Total_Amt_Chng_Q4_Q1
11. Total_Trans_Amt
12. Total_Trans_Ct
13. Total_Ct_Chng_Q4_Q1
14. Avg_Utilization_Ratio

CATEGORICAL FEATURES:
 1. Gender
 2. Education_Level
 3. Marital_Status
 4. Income_Category
 5. Card_Category

TỔNG KẾT:
- Tổng số features: 19
- Numeric features: 14
- Categorical features: 5


### 3.1 Kiểm tra Missing Values và Unknown Values

In [6]:
# Sử dụng hàm kiểm tra chất lượng dữ liệu từ module
quality_issues = dp.check_data_quality_detailed(data_clean, numeric_features, categorical_features)

KIỂM TRA CHẤT LƯỢNG DỮ LIỆU

NUMERIC FEATURES - Missing Values:
----------------------------------------
Customer_Age: Không có missing values
Dependent_count: Không có missing values
Months_on_book: Không có missing values
Total_Relationship_Count: Không có missing values
Months_Inactive_12_mon: Không có missing values
Contacts_Count_12_mon: Không có missing values
Credit_Limit: Không có missing values
Total_Revolving_Bal: Không có missing values
Avg_Open_To_Buy: Không có missing values
Total_Amt_Chng_Q4_Q1: Không có missing values
Total_Trans_Amt: Không có missing values
Total_Trans_Ct: Không có missing values
Total_Ct_Chng_Q4_Q1: Không có missing values
Avg_Utilization_Ratio: Không có missing values
Tất cả numeric features đều không có missing values!

CATEGORICAL FEATURES - Unknown/Missing Values:
--------------------------------------------------

Gender:
  Unique values: 2
  Values: ['F', 'M']
  Không có missing/unknown values

Education_Level:
  Unique values: 7
  Values: ['Coll

### 3.2 Xử lý Missing Values cho Categorical Features

In [7]:
# Thực hiện xử lý missing values
data_processed = data_clean[feature_columns]
data_with_handled_missing, imputation_info = dp.handle_categorical_missing_values(data_processed)

print(f"\n" + "="*60)
print("KẾT QUẢ XỬ LÝ MISSING VALUES")
print("="*60)

for col, info in imputation_info.items():
    print(f"\n{col}:")
    print(f"  Missing before: {info['missing_count_before']}")
    print(f"  Missing after: {info['missing_count_after']}")
    print(f"  Hoàn thành xử lý missing values cho cột {col}.")


KẾT QUẢ XỬ LÝ MISSING VALUES

Gender:
  Missing before: 0
  Missing after: 0
  Hoàn thành xử lý missing values cho cột Gender.

Education_Level:
  Missing before: 1519
  Missing after: 0
  Hoàn thành xử lý missing values cho cột Education_Level.

Marital_Status:
  Missing before: 749
  Missing after: 0
  Hoàn thành xử lý missing values cho cột Marital_Status.

Income_Category:
  Missing before: 1112
  Missing after: 0
  Hoàn thành xử lý missing values cho cột Income_Category.

Card_Category:
  Missing before: 0
  Missing after: 0
  Hoàn thành xử lý missing values cho cột Card_Category.


In [8]:
# Kiểm tra kết quả sau khi xử lý missing values
print("\n" + " KIỂM TRA LẠI SAU KHI XỬ LÝ MISSING VALUES")
print("=" * 60)

# Sử dụng lại hàm check_data_quality_detailed để kiểm tra
quality_issues_after = dp.check_data_quality_detailed(
    data_with_handled_missing, numeric_features, categorical_features
)

print(f"\nSO SÁNH TRƯỚC VÀ SAU XỬ LÝ:")
print("-" * 40)
print(f"Features có missing values trước: {len(quality_issues)}")
print(f"Features có missing values sau: {len(quality_issues_after)}")

if len(quality_issues_after) == 0:
    print("ĐÃ XỬ LÝ THÀNH CÔNG TẤT CẢ MISSING VALUES!")
else:
    print("Vẫn còn một số missing values:")
    for issue in quality_issues_after:
        print(f" {issue['column']}: {issue['missing_count']} missing")


 KIỂM TRA LẠI SAU KHI XỬ LÝ MISSING VALUES
KIỂM TRA CHẤT LƯỢNG DỮ LIỆU

NUMERIC FEATURES - Missing Values:
----------------------------------------
Customer_Age: Không có missing values
Dependent_count: Không có missing values
Months_on_book: Không có missing values
Total_Relationship_Count: Không có missing values
Months_Inactive_12_mon: Không có missing values
Contacts_Count_12_mon: Không có missing values
Credit_Limit: Không có missing values
Total_Revolving_Bal: Không có missing values
Avg_Open_To_Buy: Không có missing values
Total_Amt_Chng_Q4_Q1: Không có missing values
Total_Trans_Amt: Không có missing values
Total_Trans_Ct: Không có missing values
Total_Ct_Chng_Q4_Q1: Không có missing values
Avg_Utilization_Ratio: Không có missing values
Tất cả numeric features đều không có missing values!

CATEGORICAL FEATURES - Unknown/Missing Values:
--------------------------------------------------

Gender:
  Unique values: 2
  Values: ['F', 'M']
  Không có missing/unknown values

Educatio

In [10]:
# Cập nhật dữ liệu cho preprocessing tiếp theo
print("CẬP NHẬT DỮ LIỆU CHO PREPROCESSING")
print("=" * 50)

# Thay thế data_clean bằng data đã xử lý missing values
data_clean = data_with_handled_missing
print("Đã cập nhật data_clean với dữ liệu đã xử lý missing values")

CẬP NHẬT DỮ LIỆU CHO PREPROCESSING
Đã cập nhật data_clean với dữ liệu đã xử lý missing values


### 3.4 Feature Engineering - Tạo Features Mới

In [11]:
# Feature Engineering - Tạo 4 features số (không có categorical)
print("FEATURE ENGINEERING")
print("=" * 60)

# Tạo copy để thêm features mới
data_with_features = data_clean.copy()
print(f"Dữ liệu hiện tại: {len(data_clean.dtype.names)} columns")

# ===== 4 FEATURES SỐ =====
print("\nTẠO 4 NUMERIC FEATURES MỚI:")

# 1. Credit Utilization Efficiency (Hiệu quả sử dụng credit)
# = Credit_Limit / (Total_Revolving_Bal + 1) để tránh chia cho 0
credit_utilization_efficiency = data_clean['Credit_Limit'] / (data_clean['Total_Revolving_Bal'] + 1)

# 2. Customer Activity Score (Điểm hoạt động khách hàng)
# = (Total_Trans_Ct * Total_Trans_Amt) / (Months_on_book + 1)
customer_activity_score = (data_clean['Total_Trans_Ct'] * data_clean['Total_Trans_Amt']) / (data_clean['Months_on_book'] + 1)

# 3. Average Transaction Amount 
# = Total_Trans_Amt / Total_Trans_Ct
avg_transaction_amount = data_clean['Total_Trans_Amt'] / np.maximum(data_clean['Total_Trans_Ct'], 1)

print(f"1. Credit_Utilization_Efficiency: min = {np.min(credit_utilization_efficiency):.2f}, max = {np.max(credit_utilization_efficiency):.2f}, mean = {np.mean(credit_utilization_efficiency):.2f}")
print(f"2. Customer_Activity_Score: min = {np.min(customer_activity_score):.2f}, max = {np.max(customer_activity_score):.2f}, mean = {np.mean(customer_activity_score):.2f}")
print(f"3. Avg_Transaction_Amount: min = {np.min(avg_transaction_amount):.2f}, max =  {np.max(avg_transaction_amount):.2f}, mean = {np.mean(avg_transaction_amount):.2f}")

print(f"\nĐã tạo 3 features số mới!")

# Lưu features để sử dụng sau
engineered_numeric_features = {
    'Credit_Utilization_Efficiency': credit_utilization_efficiency,
    'Customer_Activity_Score': customer_activity_score,
    'Avg_Transaction_Amount': avg_transaction_amount
}

FEATURE ENGINEERING
Dữ liệu hiện tại: 19 columns

TẠO 4 NUMERIC FEATURES MỚI:
1. Credit_Utilization_Efficiency: min = 1.00, max = 34516.00, mean = 1965.92
2. Customer_Activity_Score: min = 98.15, max = 143891.00, mean = 10109.92
3. Avg_Transaction_Amount: min = 19.14, max =  190.19, mean = 62.61

Đã tạo 3 features số mới!


In [13]:
# Tạo structured array với features mới
print("THÊM FEATURES MỚI VÀO STRUCTURED ARRAY")
print("=" * 50)

# Tạo dtype mới từ đầu thay vì modify dtype cũ
original_names = list(data_clean.dtype.names)
original_dtypes = [data_clean.dtype[name] for name in original_names]

# Tạo list dtype mới
new_dtype_list = []
for name, dtype in zip(original_names, original_dtypes):
    new_dtype_list.append((name, dtype))

# Thêm 3 features mới
new_fields = [
    ('Credit_Utilization_Efficiency', 'f8'),  # float64
    ('Customer_Activity_Score', 'f8'),         # float64  
    ('Avg_Transaction_Amount', 'f8'),          # float64
]

new_dtype_list.extend(new_fields)

print(f"Các fields mới được thêm:")
for field_name, field_type in new_fields:
    print(f"  - {field_name} ({field_type})")

# Tạo array mới với dtype mở rộng
data_enhanced = np.zeros(len(data_clean), dtype=new_dtype_list)

# Copy dữ liệu cũ
for field in data_clean.dtype.names:
    data_enhanced[field] = data_clean[field]

# Thêm features mới
for feature_name, feature_values in engineered_numeric_features.items():
    data_enhanced[feature_name] = feature_values

print(f"\nKẾT QUẢ:")
print(f"Columns trước feature engineering: {len(data_clean.dtype.names)}")
print(f"Columns sau feature engineering: {len(data_enhanced.dtype.names)}")
print(f"Số features mới: {len(new_fields)}")

print(f"\nDanh sách tất cả columns hiện tại:")
for i, col in enumerate(data_enhanced.dtype.names, 1):
    marker = "NEW" if col in [field[0] for field in new_fields] else "   "
    print(f"{i:2d}. {marker} {col}")

# Cập nhật data_clean với data đã có features mới
data_clean = data_enhanced
print(f"\n Đã cập nhật data_clean với 3 features số mới thành công!")

THÊM FEATURES MỚI VÀO STRUCTURED ARRAY
Các fields mới được thêm:
  - Credit_Utilization_Efficiency (f8)
  - Customer_Activity_Score (f8)
  - Avg_Transaction_Amount (f8)

KẾT QUẢ:
Columns trước feature engineering: 19
Columns sau feature engineering: 22
Số features mới: 3

Danh sách tất cả columns hiện tại:
 1.     Customer_Age
 2.     Dependent_count
 3.     Months_on_book
 4.     Total_Relationship_Count
 5.     Months_Inactive_12_mon
 6.     Contacts_Count_12_mon
 7.     Credit_Limit
 8.     Total_Revolving_Bal
 9.     Avg_Open_To_Buy
10.     Total_Amt_Chng_Q4_Q1
11.     Total_Trans_Amt
12.     Total_Trans_Ct
13.     Total_Ct_Chng_Q4_Q1
14.     Avg_Utilization_Ratio
15.     Gender
16.     Education_Level
17.     Marital_Status
18.     Income_Category
19.     Card_Category
20. NEW Credit_Utilization_Efficiency
21. NEW Customer_Activity_Score
22. NEW Avg_Transaction_Amount

 Đã cập nhật data_clean với 3 features số mới thành công!


In [14]:
# Cập nhật feature lists với features mới
print("CẬP NHẬT DANH SÁCH FEATURES")
print("=" * 50)

# Cập nhật excluded columns để loại bỏ ID và target
excluded_columns_updated = ['CLIENTNUM', 'Attrition_Flag']

# Lấy lại numeric và categorical features với dữ liệu mới  
numeric_features_updated = dp.get_numeric_columns(data_clean, exclude_columns=excluded_columns_updated)
categorical_features_updated = dp.get_categorical_columns(data_clean)
categorical_features_updated = [col for col in categorical_features_updated if col not in excluded_columns_updated]

print("NUMERIC FEATURES (bao gồm 3 features mới):")
new_numeric_features = ['Credit_Utilization_Efficiency', 'Customer_Activity_Score', 'Avg_Transaction_Amount']
for i, col in enumerate(numeric_features_updated, 1):
    marker = "NEW" if col in new_numeric_features else "   "
    print(f"{i:2d}. {marker} {col}")

print(f"\nCATEGORICAL FEATURES (không thêm features mới):")  
for i, col in enumerate(categorical_features_updated, 1):
    print(f"{i:2d}.     {col}")

# Cập nhật biến để sử dụng ở các bước tiếp theo
numeric_features = numeric_features_updated  
categorical_features = categorical_features_updated

print(f"\n✅ Đã cập nhật danh sách features: thêm 3 features số mới!")
print(f"   - Tổng numeric features: {len(numeric_features)}")
print(f"   - Tổng categorical features: {len(categorical_features)}")

CẬP NHẬT DANH SÁCH FEATURES
NUMERIC FEATURES (bao gồm 3 features mới):
 1.     Customer_Age
 2.     Dependent_count
 3.     Months_on_book
 4.     Total_Relationship_Count
 5.     Months_Inactive_12_mon
 6.     Contacts_Count_12_mon
 7.     Credit_Limit
 8.     Total_Revolving_Bal
 9.     Avg_Open_To_Buy
10.     Total_Amt_Chng_Q4_Q1
11.     Total_Trans_Amt
12.     Total_Trans_Ct
13.     Total_Ct_Chng_Q4_Q1
14.     Avg_Utilization_Ratio
15. NEW Credit_Utilization_Efficiency
16. NEW Customer_Activity_Score
17. NEW Avg_Transaction_Amount

CATEGORICAL FEATURES (không thêm features mới):
 1.     Gender
 2.     Education_Level
 3.     Marital_Status
 4.     Income_Category
 5.     Card_Category

✅ Đã cập nhật danh sách features: thêm 3 features số mới!
   - Tổng numeric features: 17
   - Tổng categorical features: 5


## 4. Preprocessing Numeric Features

In [15]:
# Tạo ma trận numeric features
X_numeric = np.column_stack([data_clean[col] for col in numeric_features])

print("Thông tin numeric features:")
print("=" * 50)
for i, col in enumerate(numeric_features):
    values = X_numeric[:, i]
    print(f"{col}:")
    print(f"  - Min: {np.min(values):.2f}")
    print(f"  - Max: {np.max(values):.2f}")
    print(f"  - Mean: {np.mean(values):.2f}")
    print(f"  - Std: {np.std(values):.2f}")
    print()

print(f"Shape của ma trận numeric: {X_numeric.shape}")

# Sử dụng hàm standardize_features_with_info từ module
X_numeric_scaled, standardization_info = dp.standardize_features_with_info(X_numeric)

print("\nSau khi standardization:")
print(f"Mean của từng feature: {standardization_info['mean_of_standardized']}")
print(f"Std của từng feature: {standardization_info['std_of_standardized']}")
print(f"Features có std = 0: {standardization_info['features_with_zero_std']}")

Thông tin numeric features:
Customer_Age:
  - Min: 26.00
  - Max: 73.00
  - Mean: 46.33
  - Std: 8.02

Dependent_count:
  - Min: 0.00
  - Max: 5.00
  - Mean: 2.35
  - Std: 1.30

Months_on_book:
  - Min: 13.00
  - Max: 56.00
  - Mean: 35.93
  - Std: 7.99

Total_Relationship_Count:
  - Min: 1.00
  - Max: 6.00
  - Mean: 3.81
  - Std: 1.55

Months_Inactive_12_mon:
  - Min: 0.00
  - Max: 6.00
  - Mean: 2.34
  - Std: 1.01

Contacts_Count_12_mon:
  - Min: 0.00
  - Max: 6.00
  - Mean: 2.46
  - Std: 1.11

Credit_Limit:
  - Min: 1438.30
  - Max: 34516.00
  - Mean: 8631.95
  - Std: 9088.33

Total_Revolving_Bal:
  - Min: 0.00
  - Max: 2517.00
  - Mean: 1162.81
  - Std: 814.95

Avg_Open_To_Buy:
  - Min: 3.00
  - Max: 34516.00
  - Mean: 7469.14
  - Std: 9090.24

Total_Amt_Chng_Q4_Q1:
  - Min: 0.00
  - Max: 3.40
  - Mean: 0.76
  - Std: 0.22

Total_Trans_Amt:
  - Min: 510.00
  - Max: 18484.00
  - Mean: 4404.09
  - Std: 3396.96

Total_Trans_Ct:
  - Min: 10.00
  - Max: 139.00
  - Mean: 64.86
  - Std: 23

## 5. Preprocessing Categorical Features (One-Hot Encoding)

In [16]:
# Sử dụng hàm one_hot_encode_with_info từ module
X_categorical, categorical_feature_names, encoding_detailed_info = dp.one_hot_encode_with_info(data_clean, categorical_features)

print(f"\n{'='*60}")
print(f"TỔNG KẾT ONE-HOT ENCODING:")
print(f"- Số categorical features gốc: {len(categorical_features)}")
print(f"- Số features sau encoding: {len(categorical_feature_names)}")
print(f"- Shape ma trận categorical: {X_categorical.shape}")

ONE-HOT ENCODING CHI TIẾT:
----------------------------------------

Gender:
  Unique values (2): ['F', 'M']
  Features tạo ra: 2

Education_Level:
  Unique values (6): ['College', 'Doctorate', 'Graduate', 'High School', 'Post-Graduate', 'Uneducated']
  Features tạo ra: 6

Marital_Status:
  Unique values (3): ['Divorced', 'Married', 'Single']
  Features tạo ra: 3

Income_Category:
  Unique values (5): ['$120K +', '$40K - $60K', '$60K - $80K', '$80K - $120K', 'Less than $40K']
  Features tạo ra: 5

Card_Category:
  Unique values (4): ['Blue', 'Gold', 'Platinum', 'Silver']
  Features tạo ra: 4

TỔNG KẾT ONE-HOT ENCODING:
- Số categorical features gốc: 5
- Số features sau encoding: 20
- Shape ma trận categorical: (10127, 20)


## 6. Tạo Final Feature Matrix

In [17]:
# Kết hợp numeric và categorical features
X_final = np.column_stack([X_numeric_scaled, X_categorical])

# Tạo danh sách tên features cuối cùng
final_feature_names = numeric_features + categorical_feature_names

print("FEATURE MATRIX CUỐI CÙNG CHO LOGISTIC REGRESSION:")
print("=" * 60)
print(f"Shape của X: {X_final.shape}")
print(f"Shape của y: {target_binary.shape}")
print(f"Số features: {len(final_feature_names)}")
print(f"Số samples: {len(target_binary)}")

print(f"\nCấu trúc features:")
print(f"- Numeric features (đã chuẩn hóa): {len(numeric_features)}")
print(f"- One-hot encoded features: {len(categorical_feature_names)}")

# Kiểm tra missing values
has_nan = np.isnan(X_final).any()
print(f"\nCó missing values: {'Có' if has_nan else 'Không'}")

if has_nan:
    nan_count = np.isnan(X_final).sum()
    print(f"Tổng số missing values: {nan_count}")

FEATURE MATRIX CUỐI CÙNG CHO LOGISTIC REGRESSION:
Shape của X: (10127, 37)
Shape của y: (10127,)
Số features: 37
Số samples: 10127

Cấu trúc features:
- Numeric features (đã chuẩn hóa): 17
- One-hot encoded features: 20

Có missing values: Không


## 7. Train-Test Split

In [18]:
# Sử dụng hàm train_test_split_stratified từ module
X_train, X_test, y_train, y_test, train_idx, test_idx = dp.train_test_split_stratified(
    X_final, target_binary, test_size=0.2, random_state=42
)

print("TRAIN-TEST SPLIT HOÀN TẤT:")
print("=" * 50)
print(f"Training set:")
print(f"  - Shape: {X_train.shape}")
print(f"  - Class 0: {np.sum(y_train == 0):,} ({np.sum(y_train == 0)/len(y_train)*100:.1f}%)")
print(f"  - Class 1: {np.sum(y_train == 1):,} ({np.sum(y_train == 1)/len(y_train)*100:.1f}%)")

print(f"\nTest set:")
print(f"  - Shape: {X_test.shape}")
print(f"  - Class 0: {np.sum(y_test == 0):,} ({np.sum(y_test == 0)/len(y_test)*100:.1f}%)")
print(f"  - Class 1: {np.sum(y_test == 1):,} ({np.sum(y_test == 1)/len(y_test)*100:.1f}%)")

print(f"\nTỷ lệ train/test: {len(y_train)/len(target_binary)*100:.1f}% / {len(y_test)/len(target_binary)*100:.1f}%")

TRAIN-TEST SPLIT HOÀN TẤT:
Training set:
  - Shape: (8102, 37)
  - Class 0: 6,800 (83.9%)
  - Class 1: 1,302 (16.1%)

Test set:
  - Shape: (2025, 37)
  - Class 0: 1,700 (84.0%)
  - Class 1: 325 (16.0%)

Tỷ lệ train/test: 80.0% / 20.0%


## 8. Lưu dữ liệu đã preprocessing

In [19]:
# Lưu dữ liệu đã preprocessing
processed_data_dir = "/home/xv6/Lab 2/Credit Card Customer Analysis/data/processed"

# Tạo thư mục nếu chưa tồn tại
import os
os.makedirs(processed_data_dir, exist_ok=True)

# Lưu training data
np.save(f"{processed_data_dir}/X_train.npy", X_train)
np.save(f"{processed_data_dir}/y_train.npy", y_train)

# Lưu test data
np.save(f"{processed_data_dir}/X_test.npy", X_test)
np.save(f"{processed_data_dir}/y_test.npy", y_test)

# Lưu feature names
with open(f"{processed_data_dir}/feature_names.txt", "w") as f:
    for name in final_feature_names:
        f.write(f"{name}\n")

# Lưu preprocessing info
preprocessing_info = {
    'numeric_features': numeric_features,
    'categorical_features': categorical_features,
    'feature_names': final_feature_names,
    'train_size': len(y_train),
    'test_size': len(y_test),
    'n_features': X_train.shape[1]
}

# Lưu preprocessing info as text file (vì không có json)
with open(f"{processed_data_dir}/preprocessing_info.txt", "w") as f:
    f.write("PREPROCESSING INFORMATION\n")
    f.write("=" * 50 + "\n")
    f.write(f"Train size: {preprocessing_info['train_size']}\n")
    f.write(f"Test size: {preprocessing_info['test_size']}\n")
    f.write(f"Number of features: {preprocessing_info['n_features']}\n")
    f.write(f"Number of numeric features: {len(numeric_features)}\n")
    f.write(f"Number of categorical features: {len(categorical_features)}\n\n")
    
    f.write("NUMERIC FEATURES:\n")
    for feat in numeric_features:
        f.write(f"- {feat}\n")
    
    f.write("\nCATEGORICAL FEATURES:\n")
    for feat in categorical_features:
        f.write(f"- {feat}\n")

print("DỮ LIỆU ĐÃ ĐƯỢC LƯU THÀNH CÔNG!")
print("=" * 50)
print(f"Thư mục: {processed_data_dir}")
print("Các files đã tạo:")
print("- X_train.npy: Training features")
print("- y_train.npy: Training labels") 
print("- X_test.npy: Test features")
print("- y_test.npy: Test labels")
print("- feature_names.txt: Tên các features")
print("- preprocessing_info.txt: Thông tin preprocessing")

print(f"\nDữ liệu sẵn sàng cho Logistic Regression!")

DỮ LIỆU ĐÃ ĐƯỢC LƯU THÀNH CÔNG!
Thư mục: /home/xv6/Lab 2/Credit Card Customer Analysis/data/processed
Các files đã tạo:
- X_train.npy: Training features
- y_train.npy: Training labels
- X_test.npy: Test features
- y_test.npy: Test labels
- feature_names.txt: Tên các features
- preprocessing_info.txt: Thông tin preprocessing

Dữ liệu sẵn sàng cho Logistic Regression!


## 9. Tóm tắt Preprocessing

### Các bước đã thực hiện:

1. **Load dữ liệu** từ CSV và loại bỏ 2 cột cuối không cần thiết
2. **Tạo target binary** (0: Existing Customer, 1: Attrited Customer)
3. **Feature selection** - loại bỏ ID và target columns
4. **Numeric features preprocessing**:
   - Standardization (Z-score normalization)
   - Tất cả numeric features có mean=0, std=1
5. **Categorical features preprocessing**:
   - One-Hot Encoding cho tất cả categorical variables
   - Xử lý đúng định dạng bytes
6. **Feature matrix creation**:
   - Kết hợp numeric và categorical features
   - Kiểm tra missing values
7. **Train-Test Split**:
   - Stratified sampling để giữ tỷ lệ class
   - 80% train, 20% test
8. **Lưu trữ dữ liệu** đã preprocessing để sử dụng cho modeling

### Kết quả cuối cùng:
- **Training set**: ~8,101 samples
- **Test set**: ~2,025 samples
- **Class balance**: Được giữ nguyên trong cả train và test set
- **Features**: Đã được chuẩn hóa và encoded phù hợp cho Logistic Regression

### Sẵn sàng cho Logistic Regression Model!